## 作业要求
题目：Lending Club 数据集模型训练。

根据给定的 Lending Club 数据集进行调优，要求：

根据三种衍生变量构建的方法每个至少构建出一个衍生变量，并检验其结果；

实现非深度学习至少三个模型并进行集成；

实现至少一种深度学习网络，并比较其效果；检查是否可以将之进行集成；

根据自己选择的深度学习网络，实现至少一中训练的 trick。

参考了其他同学的作业，感谢他们的提示

## 获取数据集

In [ ]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/train_final.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test_final.csv')


X_train = df_train.drop(columns=['loan_status']).values
Y_train = df_train['loan_status'].values.astype(int)
X_test = df_test.drop(columns=['loan_status']).values
Y_test = df_test['loan_status'].values.astype(int)


Mounted at /content/drive


## lightgbm baseline


In [ ]:
def get_five_fold_data(X_train, Y_train):
  seed = 42 # for the same data division

  kf = KFold(n_splits=5, random_state=seed,shuffle=True)

  # split data for five fold

  five_fold_data = []

  for train_index, eval_index in kf.split(X_train):
      x_train, x_eval = X_train[train_index], X_train[eval_index]
      y_train, y_eval = Y_train[train_index], Y_train[eval_index]
      
      five_fold_data.append([(x_train, y_train), (x_eval, y_eval)])
  return five_fold_data

### Algorithm

In [ ]:
def get_model(param,five_fold_data ):
    model_list = []
    for idx, [(x_train, y_train), (x_eval, y_eval)] in enumerate(five_fold_data):
        print('{}-th model is training:'.format(idx))
        train_data = lgb.Dataset(x_train, label=y_train)
        validation_data = lgb.Dataset(x_eval, label=y_eval)
        bst = lgb.train(param, train_data, valid_sets=[validation_data])
        model_list.append(bst)
    return model_list

### train

In [ ]:
param_base = {'num_leaves': 31, 'objective': 'binary', 'metric': 'binary', 'num_round':1000}

param_fine_tuning = {'num_thread': 8,'num_leaves': 128, 'metric': 'binary', 'objective': 'binary', 'num_round': 1000, 
                     'learning_rate': 3e-3, 'feature_fraction': 0.6, 'bagging_fraction': 0.8}
                  

In [ ]:
# base param train
five_fold_data= get_five_fold_data(X_train, Y_train)
param_base_model = get_model(param_base, five_fold_data)

# param fine tuning
param_fine_tuning_model = get_model(param_fine_tuning,five_fold_data)

0-th model is training:


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[991]	valid_0's binary_logloss: 0.230205
[992]	valid_0's binary_logloss: 0.230293
[993]	valid_0's binary_logloss: 0.230369
[994]	valid_0's binary_logloss: 0.230437
[995]	valid_0's binary_logloss: 0.230484
[996]	valid_0's binary_logloss: 0.230528
[997]	valid_0's binary_logloss: 0.230627
[998]	valid_0's binary_logloss: 0.23072
[999]	valid_0's binary_logloss: 0.230825
[1000]	valid_0's binary_logloss: 0.230884
0-th model is training:
[1]	valid_0's binary_logloss: 0.507083
[2]	valid_0's binary_logloss: 0.505148
[3]	valid_0's binary_logloss: 0.503244
[4]	valid_0's binary_logloss: 0.501362
[5]	valid_0's binary_logloss: 0.499498
[6]	valid_0's binary_logloss: 0.497659
[7]	valid_0's binary_logloss: 0.49584
[8]	valid_0's binary_logloss: 0.494033
[9]	valid_0's binary_logloss: 0.492254
[10]	valid_0's binary_logloss: 0.491941
[11]	valid_0's binary_logloss: 0.490211
[12]	valid_0's binary_logloss: 0.488466
[13]	valid_0's binary_logloss: 0.486737
[14]	valid_0's binary_logloss

### test


In [ ]:
def test_model(model_list, X_test, Y_test):
    data = X_test
    five_fold_pred = np.zeros((5, len(X_test)))
    for i, bst in enumerate(model_list):
        ypred = bst.predict(data, num_iteration=bst.best_iteration)
        five_fold_pred[i] = ypred
    ypred_mean = (five_fold_pred.mean(axis=-2)>0.5).astype(int)
    return accuracy_score(ypred_mean, Y_test)

In [ ]:
base_score = test_model(param_base_model,X_test, Y_test)
fine_tuning_score = test_model(param_fine_tuning_model,X_test, Y_test)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

base: 0.91584, fine tuning: 0.91762


## 根据三种衍生变量构建的方法每个至少构建出一个衍生变量，并检验其结果


### 1.交叉

In [ ]:
#构造月还款能力 monthly_installment_capacity=continuous_annual_inc/12-continuous_installment

df_train_new = df_train.copy(deep=True)
df_test_new = df_test.copy(deep=True)

df_train_new['monthly_installment_capacity']= (df_train_new['continuous_annual_inc'] / 12) - df_train_new['continuous_installment']
df_test_new['monthly_installment_capacity'] = (df_test_new['continuous_annual_inc'] / 12) - df_test_new['continuous_installment']


X_train_1 = df_train_new.drop(columns=['loan_status']).values
Y_train_1 = df_train_new['loan_status'].values.astype(int)
X_test_1 = df_test_new.drop(columns=['loan_status']).values
Y_test_1 = df_test_new['loan_status'].values.astype(int)

five_fold_data_1= get_five_fold_data(X_train_1, Y_train_1)
param_base_model_1 = get_model(param_base, five_fold_data_1)

# param fine tuning
param_fine_tuning_model_1 = get_model(param_fine_tuning,five_fold_data_1)





0-th model is training:


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[6]	valid_0's binary_logloss: 0.497674
[7]	valid_0's binary_logloss: 0.495845
[8]	valid_0's binary_logloss: 0.494033
[9]	valid_0's binary_logloss: 0.492259
[10]	valid_0's binary_logloss: 0.490503
[11]	valid_0's binary_logloss: 0.490171
[12]	valid_0's binary_logloss: 0.489878
[13]	valid_0's binary_logloss: 0.488149
[14]	valid_0's binary_logloss: 0.486428
[15]	valid_0's binary_logloss: 0.484768
[16]	valid_0's binary_logloss: 0.483105
[17]	valid_0's binary_logloss: 0.481443
[18]	valid_0's binary_logloss: 0.479796
[19]	valid_0's binary_logloss: 0.479517
[20]	valid_0's binary_logloss: 0.477871
[21]	valid_0's binary_logloss: 0.476266
[22]	valid_0's binary_logloss: 0.47467
[23]	valid_0's binary_logloss: 0.473082
[24]	valid_0's binary_logloss: 0.471516
[25]	valid_0's binary_logloss: 0.469947
[26]	valid_0's binary_logloss: 0.468401
[27]	valid_0's binary_logloss: 0.466867
[28]	valid_0's binary_logloss: 0.465348
[29]	valid_0's binary_logloss: 0.465067
[30]	valid_0's bin

In [ ]:
base_score = test_model(param_base_model_1,X_test_1, Y_test_1)
fine_tuning_score = test_model(param_fine_tuning_model_1, X_test_1, Y_test_1)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

base: 0.91628, fine tuning: 0.91774


### 2.离散化

In [ ]:
corr = df_train.corr()['loan_status'].sort_values()
pd.set_option('display.max_rows', corr.shape[0]+1)
print(corr)

continuous_int_rate                      -0.309801
discrete_term_2_one_hot                  -0.236594
discrete_grade_5_one_hot                 -0.155089
discrete_grade_6_one_hot                 -0.140186
discrete_grade_3_one_hot                 -0.111315
continuous_dti                           -0.106753
continuous_inq_last_6mths                -0.098023
continuous_annual_inc_joint              -0.078364
discrete_sub_grade_11_one_hot            -0.077402
continuous_loan_amnt                     -0.075830
continuous_funded_amnt                   -0.075830
continuous_funded_amnt_inv               -0.075583
discrete_sub_grade_16_one_hot            -0.070298
discrete_sub_grade_26_one_hot            -0.069236
discrete_sub_grade_24_one_hot            -0.065462
discrete_sub_grade_28_one_hot            -0.064147
discrete_home_ownership_2_one_hot        -0.063856
discrete_sub_grade_25_one_hot            -0.063503
discrete_sub_grade_20_one_hot            -0.062807
discrete_sub_grade_13_one_hot  

In [ ]:
#continuous_int_rate最负相关变量，离散化它
df_train['continuous_int_rate'].describe()


count    50000.000000
mean        11.986125
std          4.182402
min          5.320000
25%          9.170000
50%         11.490000
75%         14.330000
max         28.990000
Name: continuous_int_rate, dtype: float64

In [ ]:
df_train['continuous_int_binning'] = pd.cut(df_train['continuous_int_rate'],[5,9,11,14,29],labels=[0,1,2,3])
df_train['continuous_int_binning'].value_counts()
df_test['continuous_int_binning'] = pd.cut(df_test['continuous_int_rate'],[5,9,11,14,29],labels=[0,1,2,3])

2    15046
3    12784
0    12441
1     9729
Name: continuous_int_binning, dtype: int64

In [ ]:
df_train['continuous_int_rate'].describe()

count    50000.000000
mean        11.986125
std          4.182402
min          5.320000
25%          9.170000
50%         11.490000
75%         14.330000
max         28.990000
Name: continuous_int_rate, dtype: float64

In [ ]:

X_train_2 = df_train.drop(columns=['loan_status','continuous_int_rate']).values
Y_train_2 = df_train['loan_status'].values.astype(int)
X_test_2 = df_test.drop(columns=['loan_status','continuous_int_rate']).values
Y_test_2 = df_test['loan_status'].values.astype(int)

five_fold_data_2= get_five_fold_data(X_train_2, Y_train_2)
param_base_model_2 = get_model(param_base, five_fold_data_2)

# param fine tuning
param_fine_tuning_model_2 = get_model(param_fine_tuning,five_fold_data_2)





0-th model is training:


/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


流式输出内容被截断，只能显示最后 5000 行内容。
[5]	valid_0's binary_logloss: 0.501065
[6]	valid_0's binary_logloss: 0.499204
[7]	valid_0's binary_logloss: 0.497366
[8]	valid_0's binary_logloss: 0.495556
[9]	valid_0's binary_logloss: 0.493754
[10]	valid_0's binary_logloss: 0.493442
[11]	valid_0's binary_logloss: 0.491699
[12]	valid_0's binary_logloss: 0.489938
[13]	valid_0's binary_logloss: 0.488188
[14]	valid_0's binary_logloss: 0.487895
[15]	valid_0's binary_logloss: 0.486181
[16]	valid_0's binary_logloss: 0.484488
[17]	valid_0's binary_logloss: 0.482801
[18]	valid_0's binary_logloss: 0.481138
[19]	valid_0's binary_logloss: 0.479479
[20]	valid_0's binary_logloss: 0.479168
[21]	valid_0's binary_logloss: 0.477552
[22]	valid_0's binary_logloss: 0.475926
[23]	valid_0's binary_logloss: 0.474362
[24]	valid_0's binary_logloss: 0.472779
[25]	valid_0's binary_logloss: 0.471205
[26]	valid_0's binary_logloss: 0.469636
[27]	valid_0's binary_logloss: 0.468084
[28]	valid_0's binary_logloss: 0.46779
[29]	valid_0's bina

In [ ]:
base_score = test_model(param_base_model_2,X_test_2, Y_test_2)
fine_tuning_score = test_model(param_fine_tuning_model_2, X_test_2, Y_test_2)

print('base: {}, fine tuning: {}'.format(base_score, fine_tuning_score))

base: 0.91608, fine tuning: 0.91728


## 实现非深度学习至少三个模型并进行集成；



In [ ]:
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

df_train = pd.read_csv('/content/drive/MyDrive/train_final.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test_final.csv')

df_train.fillna(0,inplace=True)
df_test.fillna(0,inplace=True)

X_train = df_train.drop(columns=['loan_status']).values
Y_train = df_train['loan_status'].values.astype(int)
X_test = df_test.drop(columns=['loan_status']).values
Y_test = df_test['loan_status'].values.astype(int)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### LIGHTGBM

In [ ]:
lgbm_classifier = LGBMClassifier(random_state=42)
lgbm_classifier.fit(X_train,Y_train)
lgbm_classifier.score(X_test,Y_test)

0.91768

### XGBOOST

In [ ]:
xgboost_classifier = XGBClassifier(random_state=42,n_estimators=200)
xgboost_classifier.fit(X_train,Y_train)
xgboost_classifier.score(X_test,Y_test)

0.91718

### RandomForest

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=200,random_state=42)
rf_classifier.fit(X_train,Y_train)
rf_classifier.score(X_test,Y_test)

0.91664

### Stacking

In [ ]:
from sklearn.model_selection import StratifiedKFold

def stacking(model, train_x, train_y, test_x, n_folds):
  folds = StratifiedKFold(random_state=42, n_splits=n_folds)

  pred_test_y = np.empty((test_x.shape[0],1),float) ## 模型预测的测试集y值
  pred_train_y = np.empty((0,1),float)

  for x_i,y_i in folds.split(train_x, train_y.values):
    x_train,x_val=train_x.iloc[x_i],train_x.iloc[y_i]
    y_train,y_val=train_y.iloc[x_i],train_y.iloc[y_i]
    model.fit(X=x_train,y=y_train)
    pred_train_y=np.append(pred_train_y,model.predict(x_val))
    pred_test_y=np.column_stack((pred_test_y,model.predict(test_x)))
  test_pred_a=np.mean(pred_test_y,axis=1) #按行计算均值
  return test_pred_a.reshape(-1,1),pred_train_y

In [ ]:
x_train=df_train.drop(columns=['loan_status'])
x_test=df_test.drop(columns=['loan_status'])
y_train=df_train['loan_status']

test_pred1 ,train_pred1=stacking(lgbm_classifier,x_train, y_train,x_test,10)
print(test_pred1.size)
train_pred1=pd.DataFrame(train_pred1)
test_pred1=pd.DataFrame(test_pred1)

test_pred2 ,train_pred2=stacking(xgboost_classifier,x_train, y_train,x_test, 10)
print(test_pred2.size)
train_pred2=pd.DataFrame(train_pred2)
test_pred2=pd.DataFrame(test_pred2)

test_pred3 ,train_pred3=stacking(rf_classifier,x_train, y_train,x_test, 10)
print(test_pred3.size)
train_pred3=pd.DataFrame(train_pred3)
test_pred3=pd.DataFrame(test_pred3)


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


50000


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


50000


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


50000


In [ ]:
## 将3个模型在训练集上预测的Y值，与真实的训练集Y值，再丢给模型训练
pred_train_y = pd.concat([train_pred1, train_pred2, train_pred3], axis=1)
pred_test_y = pd.concat([test_pred1, test_pred2, train_pred3], axis=1)
randomforest_classifier = RandomForestClassifier(n_estimators=200,random_state=42)
randomforest_classifier.fit(pred_train_y,y_train)
randomforest_classifier.score(pred_test_y, Y_test)

0.91762

## 实现至少一种深度学习网络，并比较其效果；检查是否可以将之进行集成；



### 检查物理配置

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change b type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  6 15:24:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.7 gigabytes of available RAM

To enable a high-RAM runtime, select the Runtime > "Change runtime type"
menu, and then select High-RAM in the Runtime shape dropdown. Then, 
re-execute this cell.


### How can we improve (save) our deep learning model.

Discretize
Variable selection

In [19]:
# coding = 'utf-8'
import numpy as np
import pandas as pd
import tqdm

def encode_label(x):
    unique=sorted(list(set([str(item) for item in np.unique(x)])))
    kv = {unique[i]: i for i in range(len(unique))}
    vfunc = np.vectorize(lambda x: kv[str(x)])
    return vfunc(x)

def encode_label_mat(x):
    _, ncol = x.shape
    result = np.empty_like(x, dtype=int)
    for col in range(ncol):
        result[:,col] = encode_label(x[:, col])
    return result

def impute_nan(x, method='median'):
    _, ncol = x.shape
    result = np.empty_like(x)

    for col in range(ncol):
        if method == 'median':
            data = x[:, col]
            impute_value = np.median(data[~pd.isnull(data) & (data != np.inf) & (data != -np.inf)])
        else:
            raise NotImplementedError()

        func = np.vectorize(lambda x: impute_value if pd.isnull(x) else x)
        result[:, col] = func(x[:, col])
    return result


def get_uniform_interval(minimum, maximum, nbins):
    result = [minimum]
    step_size = (float(maximum - minimum)) / nbins
    for index in range(nbins - 1):
        result.append(minimum + step_size * (index + 1))
    result.append(maximum)
    return result


def get_interval_v2(x, sorted_intervals):
    if pd.isnull(x):
        return -1
    if x == np.inf:
        return -2
    if x == -np.inf:
        return -3
    interval = 0
    found = False
    sorted_intervals.append(np.inf)
    while not found and interval < len(sorted_intervals) - 1:
        if sorted_intervals[interval] <= x < sorted_intervals[interval + 1]:
            return interval
        else:
            interval += 1


def get_quantile_interval(data, nbins):
    quantiles = get_uniform_interval(0, 1, nbins)
    return list(np.quantile(data[(~pd.isnull(data)) & (data != np.inf) & (data != -np.inf)], quantiles))


def discretize(x, nbins=20):
    nrow, ncol = x.shape
    result = np.empty_like(x)
    interval_list = list()
    for col in range(ncol):
        intervals = sorted(list(set(get_quantile_interval(x[:, col], nbins))))
        interval_centroid = list()

        for i in range(len(intervals) - 1):
            interval_centroid.append(0.5 * (intervals[i] + intervals[i + 1]))
        func = np.vectorize(lambda x: get_interval_v2(x, intervals))
        result[:, col] = encode_label(func(x[:, col]))
        interval_list.append(interval_centroid)
    return result.astype(np.int64), interval_list

def get_var_type(df):
    columns = df.columns
    continuous_vars = [x for x in columns if x.startswith('continuous_')]
    discrete_vars = [x for x in columns if x.startswith('discrete_')]
    other_vars = list()
    for column in columns:
        if column not in continuous_vars and column not in discrete_vars:
            other_vars.append(column)
    return {'continuous': continuous_vars,
            'discrete': discrete_vars,
            'other': other_vars}


def get_cont_var(df):
    var_types = get_var_type(df)
    return var_types['continuous']


def get_dis_var(df):
    var_types = get_var_type(df)
    return var_types['discrete']

def drop_const_var(data):
    result = data.copy(deep=True)
    for col in data.columns:
        if len(data.loc[~pd.isnull(data[col]), col].unique()) <= 1:
            result.drop(columns=col, inplace=True)
    return result

### Load Data

In [20]:
import torch
from torch.utils.data import Dataset, DataLoader

class MyDataSet(Dataset):
    def __init__(self, x, y):
        super().__init__()
        self.x = x
        self.y = y
        self.len = x.shape[0]
        
    def __getitem__(self, idx):
        return self.x[idx, :], self.y[idx]
    
    def __len__(self):
        return self.len

In [21]:
import numpy as np
import pandas as pd
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive',force_remount=True)

df_train = pd.read_csv('/content/drive/MyDrive/train_final.csv')
df_test = pd.read_csv('/content/drive/MyDrive/test_final.csv')

x_train = df_train.drop(columns=['loan_status'])
y_train = df_train['loan_status']
x_test = df_test.drop(columns=['loan_status'])
y_test = df_test['loan_status']


num_workers = 4

x_train_ = torch.from_numpy(impute_nan(x_train.values)).type(torch.float32)
y_train_ = torch.from_numpy(y_train.values).type(torch.int)
x_test_, y_test_ = torch.from_numpy(impute_nan(x_test.values)).type(torch.float32), torch.from_numpy(y_test.values).type(torch.int)

train_dataset = MyDataSet(x_train_, y_train_)
test_dataset = MyDataSet(x_test_, y_test_)
train_dataloader = DataLoader(train_dataset, batch_size = 200, shuffle=True, num_workers=num_workers)
test_dataloader = DataLoader(test_dataset, batch_size = 200, num_workers=num_workers)


Mounted at /content/drive


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [22]:
import torch.nn as nn
import torch
import torch.nn.functional as F


def mish(input):

    return input * torch.tanh(F.softplus(input))

class Mish(nn.Module):

    def __init__(self):
        '''
        Init method.
        '''
        super().__init__()

    def forward(self, input):
        '''
        Forward pass of the function.
        '''
        return mish(input)

class MLPLayer(nn.Module):
    def __init__(self, dim_in, dim_out, res_coef = 0, dropout_p = 0.1):
        super().__init__()
        self.linear  = nn.Linear(dim_in, dim_out)
        self.res_coef = res_coef
        self.activation = Mish()
        self.dropout = nn.Dropout(dropout_p)
        self.ln = nn.LayerNorm(dim_out)
    
    def forward(self, x):
        y = self.linear(x)
        y = self.activation(y)
        y = self.dropout(y)
        if self.res_coef == 0:
            return self.ln(y)
        else:
            return self.ln(self.res_coef*x +y )

       
class MyNetwork(nn.Module):
    def __init__(self, dim_in, dim, res_coef=0.5, dropout_p = 0.1, n_layers = 10):
        super().__init__()
        self.mlp = nn.ModuleList()
        self.first_linear = MLPLayer(dim_in, dim)
        self.n_layers = n_layers
        for i in range(n_layers):
            self.mlp.append(MLPLayer(dim, dim, res_coef, dropout_p))
        self.final = nn.Linear(dim, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.first_linear(x)
        for layer in self.mlp:
            x = layer(x)
        x= self.sigmoid(self.final(x))
        return x.squeeze()

In [8]:
!pip install pytorch_lightning

     |████████████████████████████████| 849kB 15.4MB/s 
     |████████████████████████████████| 112kB 31.6MB/s 
     |████████████████████████████████| 184kB 30.9MB/s 
     |████████████████████████████████| 829kB 31.3MB/s 
     |████████████████████████████████| 276kB 34.8MB/s 
     |████████████████████████████████| 1.3MB 53.2MB/s 
     |████████████████████████████████| 143kB 55.8MB/s 
     |████████████████████████████████| 296kB 57.8MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491058 sha256=c7c42ea7450651b1aec136719a2b4d25fcd8e732f32e1c9d438d825ea15a4741
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for PyYAML: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=d972ea2d6022aaad64b12f74796def912fa48b6d216492ce142fe2148347ec1c
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built futur

In [23]:
import pytorch_lightning as pl
from pytorch_lightning.metrics import Accuracy
class TrainingModule(pl.LightningModule):
    def __init__(self, dim_in, dim, res_coef=0, dropout_p=0, n_layers=10):
        super().__init__()
        self.backbone = MyNetwork(dim_in, dim, res_coef, dropout_p, n_layers)
        self.loss = nn.BCELoss()
        self.accuracy = Accuracy()
    def forward(self, x):
        return self.backbone(x)
    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = self.backbone(x)
        loss = self.loss(x, y.type(torch.float32))
        acc = self.accuracy(x, y)
        self.log("Validation loss", loss)
        self.log("Validation acc", acc)
        return loss, acc
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = self.backbone(x)
        loss = self.loss(x, y.type(torch.float32))
        acc = self.accuracy(x, y)
        self.log("Training loss", loss)
        self.log("Training acc", acc)
        
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

import os
class CheckpointEveryNSteps(pl.Callback):
    def __init__(self, save_step_frequency):
        self.save_step_frequency = save_step_frequency

    def on_batch_end(self, trainer: pl.Trainer, _):
        epoch = trainer.current_epoch
        global_step = trainer.global_step
        if global_step % self.save_step_frequency == 0:
            filename = "epoch=" + str(epoch) + "_step=" + str(global_step)+".ckpt"
            ckpt_path = os.path.join(trainer.checkpoint_callback.dirpath, filename)
            trainer.save_checkpoint(ckpt_path)

### Train model


In [ ]:
from pytorch_lightning import loggers as pl_loggers

tb_logger = pl_loggers.TensorBoardLogger('logs/')
save_by_steps = CheckpointEveryNSteps(100)
training_module = TrainingModule(x_train.shape[1], 50, 0.5, 0.1, 30)
trainer = pl.Trainer(max_epochs=100, gpus=0, progress_bar_refresh_rate=100, val_check_interval=0.25, logger=tb_logger)
trainer.fit(training_module, train_dataloader, test_dataloader)

GPU available: True, used: False
TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: GPU available but not used. Set the --gpus flag when calling the script.
  warnings.warn(*args, **kwargs)

  | Name     | Type      | Params
---------------------------------------
0 | backbone | MyNetwork | 87.0 K
1 | loss     | BCELoss   | 0     
2 | accuracy | Accuracy  | 0     
---------------------------------------
87.0 K    Trainable params
0         Non-trainable params
87.0 K    Total params
0.348     Total estimated model params size (MB)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# 根据自己选择的深度学习网络，实现至少一中训练的 trick。